# multiprocessing
importing required libraries and our shared library

In [ ]:
import ctypes
import multiprocessing
import os
import time

In [ ]:
''' COMPILING and LINKING the code
$ gcc -c -Wall -Werror -fpic main.c
$ gcc -shared -o libMyLib.so main.o
'''
_libInC = ctypes.CDLL('./libMyLib.so')#libMyLib.so this file is generated during LINKING-PROCESS

Here, we slightly adjust our Python wrapper to calculate the results and print it. There is also some additional casting to ensure that the result of the _libInC.myAdd()_ is an int32 type.

In [ ]:
def addC_print(_i, a, b, time_started):
    val = ctypes.c_int32(_libInC.myAdd(a, b)).value #cast the result to a 32 bit integer
    end_time = time.time()
    print('CPU_{} Add: {} in {}'.format(_i, val, end_time - time_started))
    
def multC_print(_i, a, b, time_started):
    val = ctypes.c_int32(_libInC.myMult(a, b)).value #cast the result to a 32 bit integer
    end_time = time.time()
    print('CPU_{} Multiply: {} in {}'.format(_i, val, end_time - time_started))

Now for the fun stuff.

The multiprocessing library allows us to run simultaneous code by utilizing multiple processes. These processes are handled in separate memory spaces and are not restricted to the Global Interpreter Lock (GIL). 

Here we define two proceses, one to run the _addC_print_ and another to run the _multC_print()_ wrappers. 

Next we assign each process to be run on difference CPUs

In [ ]:
procs = [] # a future list of all our processes

# Launch process1 on CPU0
p1_start = time.time()
p1 = multiprocessing.Process(target=addC_print, args=(0, 3, 5, p1_start)) # the first arg defines which CPU to run the 'target' on
os.system("taskset -p -c {} {}".format(0, p1.pid)) # taskset is an os command to pin the process to a specific CPU
p1.start() # start the process
procs.append(p1)

# Launch process2 on CPU1
p2_start = time.time()
p2 = multiprocessing.Process(target=multC_print, args=(1, 3, 5, p2_start)) # the first arg defines which CPU to run the 'target' on
os.system("taskset -p -c {} {}".format(1, p2.pid)) # taskset is an os command to pin the process to a specific CPU
p2.start() # start the process
procs.append(p2)

p1Name = p1.name # get process1 name
p2Name = p2.name # get process2 name

# Here we wait for process1 to finish then wait for process2 to finish
p1.join() # wait for process1 to finish
print('Process 1 with name, {}, is finished'.format(p1Name))

p2.join() # wait for process2 to finish
print('Process 2 with name, {}, is finished'.format(p2Name))

Return to 'main.c' and change the amount of sleep time (in seconds) of each function.

For different values of sleep(), explain the difference between the results of the ‘Add’ and ‘Multiply’ functions and when the Processes are finished.

## RLS notes
                 taskset is an os command to pin the process to a specific CPU
root@pynq:/home/xilinx/jupyter_notebooks/RLS/Lab2# taskset -h
Usage: taskset [options] [mask | cpu-list] [pid|cmd [args...]]
Show or change the CPU affinity of a process.

Options:
 -a, --all-tasks         operate on all the tasks (threads) for a given pid
 -p, --pid               operate on existing given pid
 -c, --cpu-list          display and specify cpus in list format
 -h, --help              display this help
 -V, --version           display version

The default behavior is to run a new command:
    taskset 03 sshd -b 1024
You can retrieve the mask of an existing task:
    taskset -p 700
Or set it:
    taskset -p 03 700
List format uses a comma-separated list instead of a mask:
    taskset -pc 0,3,7-11 700
Ranges in list format can take a stride argument:
    e.g. 0-31:2 is equivalent to mask 0x55555555

For more details see taskset(1).

In [ ]:
## My Test code: Multiprocessing, linear programming
import ctypes
import multiprocessing
import os
import time

''' COMPILING and LINKING the code
$ gcc -c -Wall -Werror -fpic main.c
$ gcc -shared -o libMyLib.so main.o
'''
_libInC = ctypes.CDLL('./libMyLib.so')#libMyLib.so this file is generated during LINKING-PROCESS

def addC_print(_i, a, b, time_started):
    val = ctypes.c_int32(_libInC.myAdd(a, b)).value #cast the result to a 32 bit integer
    end_time = time.time()
    print('.', end='')
    print('CPU_{} Add: {} in {}'.format(_i, val, end_time - time_started))
    
def multC_print(_i, a, b, time_started):
    val = ctypes.c_int32(_libInC.myMult(a, b)).value #cast the result to a 32 bit integer
    end_time = time.time()
    print('.', end='')
    print('CPU_{} Multiply: {} in {}'.format(_i, val, end_time - time_started))

procs = [] # a future list of all our processes

# Launch process1 on CPU0
cpu1 = 0;
a = 3;b = 5
p1_start = time.time()
p1 = multiprocessing.Process(target=addC_print, args=(cpu1, a, b, p1_start)) # the first arg defines which CPU to run the 'target' on
os.system("taskset -p -c {} {}".format(0, p1.pid)) # taskset is an os command to pin the process to a specific CPU
p1.start() # start the process
procs.append(p1)

# Launch process2 on CPU1
cpu2 = 1;
a = 3;b = 5
p2_start = time.time()
p2 = multiprocessing.Process(target=multC_print, args=(cpu2, a, b, p2_start)) # the first arg defines which CPU to run the 'target' on
os.system("taskset -p -c {} {}".format(1, p2.pid)) # taskset is an os command to pin the process to a specific CPU
p2.start() # start the process
procs.append(p2)

p1Name = p1.name # get process1 name
p2Name = p2.name
p1PID = p1.pid # get process1 PID
p2PID = p2.pid

# Here we wait for process1 to finish then wait for process2 to finish
p1.join() # wait for process1 to finish, join(): Block until all items in the queue have been gotten and processed.
print('Process 1 with name: {}, PID: {} is finished'.format(p1Name, p1PID))#;print('PID {}'.format(p1PID))

p2.join() # wait for process2 to finish
print('Process 2 with name: {}, PID: {} is finished'.format(p2Name, p2PID))#;print('PID {}'.format(p2PID))



In [ ]:
## My Test code: Multiprocessing Array object
import ctypes
import multiprocessing
import os
import time
''' COMPILING and LINKING the code
$ gcc -c -Wall -Werror -fpic main.c
$ gcc -shared -o libMyLib.so main.o
'''
_libInC = ctypes.CDLL('./libMyLib.so')#libMyLib.so this file is generated during LINKING-PROCESS

def addC_print(_i, a, b, time_started):
    val = ctypes.c_int32(_libInC.myAdd(a, b)).value #cast the result to a 32 bit integer
    end_time = time.time()
    print('CPU_{} Add: {} in {}'.format(_i, val, end_time - time_started))
    
def multC_print(_i, a, b, time_started):
    val = ctypes.c_int32(_libInC.myMult(a, b)).value #cast the result to a 32 bit integer
    end_time = time.time()
    print('CPU_{} Multiply: {} in {}'.format(_i, val, end_time - time_started))

a = 3;b = 5
cpusAry=[0, 1]
functionsAry = [addC_print, multC_print]
procs = []
for i in range(2):#0,1
    process_start = time.time()
    p = multiprocessing.Process(target=functionsAry[i], args=(cpusAry[i], a, b, process_start))
    os.system("taskset -p -c {} {}".format(cpusAry[i], p.pid)) # taskset is an os command to pin the process to a specific CPU
    p.start() # start the process
    procs.append(p)
    print('Process: {}, PID: {} Started'.format(p.name, p.pid))
    
procs[0].join() # wait for process1 to finish, join(): Block until all items in the queue have been gotten and processed.
print('Process 1 with name: {}, PID: {} is finished'.format(procs[0].name, procs[0].pid))#;print('PID {}'.format(p1PID))

procs[1].join() # wait for process2 to finish
print('Process 2 with name: {}, PID: {} is finished'.format(procs[0].name, procs[0].pid))

In [ ]:

cpusAry=[0, 1]
functionsAry = [addC_print, multC_print]
for i in range(2):
    print(i)
    print(functionsAry[i])
    print(cpusAry[i])
    process_start = time.time()
    print(process_start)

# Lab work

One way around the GIL in order to share memory objects is to use multiprocessing objects. Here, we're going to do the following.

1. Create a multiprocessing array object with 2 entries of integer type.
2. Launch 1 process to compute addition and 1 process to compute multiplication.
3. Assign the results to separate positions in the array.
  1. Process 1 (add) is stored in index 0 of the array (array[0])
  2. Process 2 (mult) is stored in index 1 of the array (array[1])
4. Print the results from the array.


Thus, the multiprocessing Array object exists in a _shared memory_ space so both processes can access it.

## Array documentation:

https://docs.python.org/2/library/multiprocessing.html#multiprocessing.Array


## typecodes/types for Array:
'c': ctypes.c_char

'b': ctypes.c_byte

'B': ctypes.c_ubyte

'h': ctypes.c_short

'H': ctypes.c_ushort

'i': ctypes.c_int

'I': ctypes.c_uint

'l': ctypes.c_long

'L': ctypes.c_ulong

'f': ctypes.c_float

'd': ctypes.c_double

## Try to find an example

You can use online reources to find an example for how to use multiprocessing Array 

In [10]:
## My Test code: Multiprocessing Array object
import ctypes
import multiprocessing
import os
import time
from multiprocessing import Process, Lock
from multiprocessing.sharedctypes import Value, Array
from ctypes import Structure, c_double
''' COMPILING and LINKING the code
$ gcc -c -Wall -Werror -fpic main.c
$ gcc -shared -o libMyLib.so main.o
'''
_libInC = ctypes.CDLL('./libMyLib.so')#libMyLib.so this file is generated during LINKING-PROCESS
def addC_no_print(_i, a, b, returnValue):
    '''
    Params:
      _i   : Index of the process being run (0 or 1)
      a, b : Integers to add
      returnValues : Multiprocessing array in which we will store the result at index _i
    '''
    val = ctypes.c_int32(_libInC.myAdd(a, b)).value
    # TODO: add code here to pass val to correct position returnValues
    returnValues[_i] = val
    
def multC_no_print(_i, a, b, returnValue):
    '''
    Params:
      _i   : Index of the process being run (0 or 1)
      a, b : Integers to multiply
      returnValues : Multiprocessing array in which we will store the result at index _i
    '''    
    val = ctypes.c_int32(_libInC.myMult(a, b)).value
    print("value = "+str(val))
    # TODO: add code here to pass val to correct position of returnValues
    returnValues[_i] = val
    
procs = []

# TODO: define returnValues here. Check the multiprocessing docs to see 
# about initializing an array object for 2 processes. 
# Note the data type that will be stored in the array
returnValues = Array(c_double,[0, 0], lock=False)

cpusAry=[0, 1];a = 3;b = 5;functionsAry = [addC_no_print, multC_no_print]
for i in range(2):#0,1
    process_start = time.time()
    p = multiprocessing.Process(target=functionsAry[i], args=(cpusAry[i], a, b, returnValues))
    os.system("taskset -p -c {} {}".format(cpusAry[i], p.pid)) # taskset is an os command to pin the process to a specific CPU
    p.start() # start the process
    procs.append(p)
    print('Process: {}, PID: {} Started'.format(p.name, p.pid))
# Wait for the processes to finish
for p in procs:
    pName = p.name # get process name
    p.join() # wait for the process to finish
    print('{} is finished'.format(pName))
# TODO print the results that have been stored in returnValues
for i in range(2):
    print('Return Value {} = {}'.format(i, returnValues[i]))

taskset: invalid PID argument: 'None'


Process: Process-17, PID: 4961 Started


taskset: invalid PID argument: 'None'


Process: Process-18, PID: 4964 Started
Process-17 is finished
value = 15
Process-18 is finished
Return Value 0 = 8.0
Return Value 1 = 15.0
